In [1]:
import os
from openai import AzureOpenAI

In [ ]:
import os

os.environ["AZURE_OPENAI_API_KEY"] = "d3116cc2cd9c47fda7474f67aa272385"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://msec-cognitive-sb-eval02-oai.openai.azure.com"

In [56]:
from langchain_openai import AzureOpenAIEmbeddings

In [58]:
embeddings = AzureOpenAIEmbeddings(
    azure_deployment="AA-text-embedding-3-large",
    openai_api_version="2024-02-01",
)

In [59]:
from langchain_community.document_loaders import TextLoader

In [60]:
file_path = 'data/manufacturing-unit1/auto/manufacturing-unit1.md'
loader = TextLoader(file_path)
docs = loader.load()
print(len(docs))

1


In [61]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [62]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n#\s*\w+"], is_separator_regex=True, chunk_size=4000, chunk_overlap=200)
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=200)


In [63]:
docs[0].page_content

'SECONDEDITION  \n\n# MANUFACTURING PROCESSES  \n\nH.N. Gupta · R.C. Gupta · Arun Mittal  \n\n# MANUFACTURING PROCESSES  \n\nThis page   \nintentionally left   \nblank  \n\n# MANUFACTURING PROCESSES  \n\n(SECOND EDITION)  \n\n# H.N. Gupta  \n\nB.Sc., G.l. Mech.E (London), FIE VisitingProfessor   \nDepartment of Mechanical Engineering   \n1.E.T., Lucknow, U.P. Technical University R.C. Gupta B.Sc., B.E., M.Tech., Ph.D. ProfessorandHead   \nDepartment of Mechanical Engineering   \n1.E.T., Lucknow, U.P. Technical University  \n\n# Arun Mittal  \n\nSeniorFaculty Department of Mechanical Engineering I.E.T., Lucknow, U.P. Technical University  \n\nNEW AGE INTERNATIONAL (P) LIMITED, PUBLISHERS New Delhi $\\bullet$ Bangalore $\\bullet$ Chennai $\\bullet$ Cochin $\\bullet$ Guwahati $\\cdot$ Hyderabad Jalandhar $\\bullet$ Kolkata $\\bullet$ Lucknow $\\bullet$ Mumbai $\\bullet$ Ranchi Visit us at www.newagepublishers.com  \n\nCopyright $\\copyright$ 2009, New Age International (P) Ltd., Publisher

In [ ]:
splits = text_splitter.split_documents(docs)

In [65]:
len(splits)

21

In [67]:
from langchain_chroma import Chroma

In [69]:
db = Chroma.from_documents(splits, embeddings, persist_directory="./chroma_storage")

In [77]:

db = Chroma(persist_directory="./chroma_storage", embedding_function=embeddings)  # Load from storage


In [78]:
query = "What is the stress-strain curve of a ductile material?"
docs = db.similarity_search(query)

In [79]:
docs

[Document(metadata={'source': 'data/manufacturing-unit1/auto/manufacturing-unit1.md'}, page_content='\n# STRESS-STRAIN DIAGRAM  \n\nConsider a rod of initial length $L_{0}$ and area $A_{0}$ which is subjected to a load $F_{\\mathrm{.}}$ The stress $\\upsigma$ is the force per unit area, and strain ε is the change in length (8) divided by the initial length. Thus,  \n\n$$\n\\begin{array}{r}{\\mathrm{Stress}\\ \\sigma=F/A_{0}}\\\\ {\\mathrm{Strain}\\ \\varepsilon=\\delta/L_{0}}\\end{array}\n$$  \n\nThe O-e curve for a material (say mild steel) is shown in the Fig. 1.1. Up to the proportionality point $A$ , the stress-strain variation is linear. Up to this point Hooke\'s law holds good.  \n\ni.e., $\\begin{array}{l}{\\mathtt{C}\\propto\\mathtt{E}}\\\\ {\\upsigma=E\\mathrm{\\mathrm{g}}}\\end{array}$ or  \n\nwhere $E$ is the Young\'s modulus commonly called modulus of elasticity.  \n\nBeyond point $A$ and up to point $B$ , material remains elastic i.e., the material returns to its original 

In [84]:
def add_text_if_not_exists(text, metadata, db):
  docs = db.similarity_search(text, k=1)
  if docs[0].page_content == text:
    print("Text already exists in the database")
    return False
  db.add_texts([text], [metadata])
  print("Text added to the database")
  return True

def add_docs_if_not_exists(docs, db):
  for doc in docs:
    add_text_if_not_exists(doc.page_content, doc.metadata, db)


In [85]:
add_docs_if_not_exists(docs, db)

Text already exists in the database
Text already exists in the database
Text already exists in the database
Text already exists in the database
